# zustand

## 是什么

zustand 是一个状态管理库

## 解决了什么

解决状态同步问题

## 它是如何解决这个问题的

In [ ]:
/**
 * 这个方法返回一个闭包，保证了state,listeners的存活状态
 * 几个方法都使用箭头函数，保证能正确获取到相应的值
 * 
 * createState 形如
 * (setState,getState,subscribe,destroy)=>({
 *   xx:0,
 *   fn:()=>{
 *      // 在这里可以直接调用上面传过来的4个方法去更新闭包中的状态
 *   }
 * })
 */

const createStoreImpl = (createState)=>{
    let state;
    let listeners = new Set();

    const setState = (partial, replace)=>{
        const nextState = typeof partial === 'function' ? partial(state) : partial;
        if(!Object.is(nextState, state)){
            state = replace ? nextState : Object.assign({}, state, nextState);
            listeners.forEach(listener => listener(state));
        }
    }

    const getState = ()=> state;

    const subscribe = (listener)=>{
        listeners.add(listener);
        return ()=>listeners.delete(listener)
    }

    const destroy = ()=>{
        listeners.clear();
    }

    const api = {setState, getState, subscribe, destroy};
    state = createState(setState, getState, subscribe, destroy);

    return api;
}


/**
 * 这一步是如果调用create时传递了方法就说明是要生成一个store,
 * 否则就直接返回创建方法
 */
const createStore = (createState)=>{
    createState? createStoreImpl(createState): createStoreImpl;
}


## 如何在react 中应用

`import useSyncExternalStoreExports from 'use-sync-external-store/shim/with-select`

主要是使用了react 官方提供的一个可以使用外部状态管理的库